In [10]:
# To generate GIFs
%pip install imageio
%pip install git+https://github.com/tensorflow/docs


[notice] A new release of pip is available: 24.1 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/tensorflow/docs to /private/var/folders/p_/j8lh2l8d3r3dwh95sx4bxlt40000gn/T/pip-req-build-fc4gxwix
  Running command git clone --filter=blob:none --quiet https://github.com/tensorflow/docs /private/var/folders/p_/j8lh2l8d3r3dwh95sx4bxlt40000gn/T/pip-req-build-fc4gxwix
  Resolved https://github.com/tensorflow/docs to commit 6680535155460f7eb0d2d615b9749a0cf721d4ec
  Preparing metadata (setup.py) ... done
  Using cached astor-0.8.1-py2.py3-none-any.whl.metadata (4.2 kB)
Using cached astor-0.8.1-py2.py3-none-any.whl (27 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 1.6 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 5.2 MB/s eta 0:00:00
   ━━━━

In [12]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow.keras import layers
import time

from IPython import display

In [13]:
(train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()

In [14]:
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype("float32")
train_images = (train_images - 127.5) / 127.5 # Normalizes images to [-1,1]

In [15]:
BUFFER_SIZE = 60000
BATCH_SIZE = 256

In [16]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)